In [4]:
# -*-coding:utf8-*-
import re
from imp import reload
import string
import sys
import os
import urllib
import urllib.request
from bs4 import BeautifulSoup
import requests
from lxml import etree
import json

#定义页面打开函数
proxy_addr="122.241.72.191:808"
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data = use_proxy(url, proxy_addr)
    content = json.loads(data).get('data')
    # for key, values in content.get('tabsInfo').get('tabs').items():          只是取了键，没有取到值
    if content != None and content.get('tabsInfo') != None:
        for info in content.get('tabsInfo').get('tabs'):
            if info.get('tab_type') == 'weibo':
                containerid = info.get('containerid')
                return containerid

user_id = input("请输入user_id: ")
i = 1
url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value='+user_id
weibo_url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value='+user_id+'&containerid='+get_containerid(url)+'&page='+str(i)

html = requests.get(weibo_url).content
selector = etree.HTML(html)

# 1223178222
# pageNum = int(selector.xpath('//input[@name="mp"]')[0].attrib['value'])
pageNum = 2
result = ""
urllist_set = set()
word_count = 1
image_count = 1

print('爬虫准备就绪...')

for page in range(1, pageNum + 1):
    # 获取lxml页面
    lxml = requests.get(weibo_url).content
    print(lxml)
    # 文字爬取
    selector = etree.HTML(lxml)
    content = selector.xpath('//span[@class="ctt"]')
    for each in content:
        text = each.xpath('string(.)')
        if word_count >= 4:
            text = "%d :" % (word_count - 3) + text + "\n\n"
        else:
            text = text + "\n\n"
        result = result + text
        word_count += 1

    # 图片爬取
    soup = BeautifulSoup(lxml, "lxml")
    urllist = soup.find_all('a', href=re.compile(r'^http://weibo.cn/mblog/oripic', re.I))
    first = 0
    for imgurl in urllist:
        urllist_set.add(requests.get(imgurl['href'], cookies=cookie).url)
        image_count += 1

fo = open("1223178222.txt", "wb")
fo.write(result.encode(encoding="utf-8"))
word_path = os.getcwd() + '\%s' % user_id
print('文字微博爬取完毕')

link = ""
fo2 = open("1223178222_imageurls.txt", "wb")
for eachlink in urllist_set:
    link = link + eachlink + "\n"
fo2.write(link.encode(encoding="utf-8"))
print('图片链接爬取完毕')

if not urllist_set:
    print('该页面中不存在图片')
else:
    # 下载图片,保存在当前目录的pythonimg文件夹下
    image_path = os.getcwd() + '\weibo_image'
    print("     ",image_path)
    if os.path.exists(image_path) is False:
        os.mkdir(image_path)
    x = 1
    for imgurl in urllist_set:
        temp = image_path + '\%s.jpg' % x
        print('正在下载第%s张图片' % x)
        try:
            urllib.urlretrieve(urllib.request.urlopen(imgurl).geturl(), temp)
        except:
            print("该图片下载失败:%s" % imgurl)
        x += 1

print('原创微博爬取完毕，共%d条，保存路径%s' % (word_count - 4, word_path))
print('微博图片爬取完毕，共%d张，保存路径%s' % (image_count - 1, image_path))

请输入user_id: 1223178222
    https://m.weibo.cn/api/container/getIndex?type=uid&value=1223178222&containerid=1076031223178222&page=1
爬虫准备就绪...
b'{"ok":1,"data":{"cardlistInfo":{"containerid":"1076031223178222","v_p":42,"show_style":1,"total":3701,"page":2},"cards":[{"card_type":9,"itemid":"1076031223178222_-_4346276986742017","scheme":"https:\\/\\/m.weibo.cn\\/status\\/HjpSishUd?mblogid=HjpSishUd&luicode=10000011&lfid=1076031223178222","mblog":{"created_at":"03-05","id":"4346276986742017","idstr":"4346276986742017","mid":"4346276986742017","can_edit":false,"show_additional_indication":0,"text":"\\u4f60\\u771f\\u7684\\u5f88\\u50cf\\u6211\\u5c0f\\u5b66\\u540c\\u5b66 \\u6211\\u8ba9\\u4ed6\\u4e00\\u96c6\\u4e0d\\u843d<span class=\\"url-icon\\"><img alt=[\\u5141\\u60b2] src=\\"\\/\\/h5.sinaimg.cn\\/m\\/emoticon\\/icon\\/default\\/d_yunbei-c6964bf237.png\\" style=\\"width:1em; height:1em;\\" \\/><\\/span>\\/\\/<a href=\'\\/n\\/\\u8042\\u5c0f\\u95fb\'>@\\u8042\\u5c0f\\u95fb<\\/a>:\\u7b2c\\u4e00\

b'{"ok":1,"data":{"cardlistInfo":{"containerid":"1076031223178222","v_p":42,"show_style":1,"total":3701,"page":2},"cards":[{"card_type":9,"itemid":"1076031223178222_-_4346276986742017","scheme":"https:\\/\\/m.weibo.cn\\/status\\/HjpSishUd?mblogid=HjpSishUd&luicode=10000011&lfid=1076031223178222","mblog":{"created_at":"03-05","id":"4346276986742017","idstr":"4346276986742017","mid":"4346276986742017","can_edit":false,"show_additional_indication":0,"text":"\\u4f60\\u771f\\u7684\\u5f88\\u50cf\\u6211\\u5c0f\\u5b66\\u540c\\u5b66 \\u6211\\u8ba9\\u4ed6\\u4e00\\u96c6\\u4e0d\\u843d<span class=\\"url-icon\\"><img alt=[\\u5141\\u60b2] src=\\"\\/\\/h5.sinaimg.cn\\/m\\/emoticon\\/icon\\/default\\/d_yunbei-c6964bf237.png\\" style=\\"width:1em; height:1em;\\" \\/><\\/span>\\/\\/<a href=\'\\/n\\/\\u8042\\u5c0f\\u95fb\'>@\\u8042\\u5c0f\\u95fb<\\/a>:\\u7b2c\\u4e00\\u6b21\\u505a\\u5236\\u7247\\u4eba\\u7684\\u620f\\uff0c\\u867d\\u7565\\u6709\\u9057\\u61be\\u4f46\\u603b\\u5f52\\u662f\\u503e\\u5c3d\\u5fc3\\

NameError: name 'image_path' is not defined

In [1]:
import requests
import os
import re
import traceback
from lxml import etree
from bs4 import BeautifulSoup

def get_html(url,cookies):
    try:
        re = requests.get(url,cookies=cookies)
        re.raise_for_status
    except:
        return ""
    return re.content    

def get_comment(lxml,count):
    text=""
    print("开始爬取文字微博")
    fpath = "./微博爬虫.txt"
    html = etree.HTML(lxml)
    all_comments = html.xpath('//span[@class="ctt"]')
    for each in all_comments:
    #    try:
            comment =str(count)+" "+ each.xpath('string(.)')+"\n\n"
            text= text + comment
            count+=1
      #  except:
        #    continue

    with open(fpath,'a',encoding="utf-8") as f:
        f.write(text)
        print("文字微博爬取完成")

def get_img(lxml):
    root = "./微博图片//"
    print("开始爬取图片微博")
    html = BeautifulSoup(lxml,"lxml")
    taglist = html.find_all('img',attrs={"alt":"图片"})
    for i in range(len(taglist)):
   #  try:
           imgurl = taglist[i]["src"]
           fpath = root + imgurl.split('/')[-1]
           imgurl=imgurl.replace('wap180','large')
           print(imgurl)
           content = requests.get(imgurl).content
           with open(fpath,'wb') as f:
               f.write(content)
               print("%d爬取成功")%imgurl
   #  except:
     #      continue

def main():
    cookies={"Cookie":"SINAGLOBAL=1938570347430.7502.1526912194200; TC-V5-G0=28bf4f11899208be3dc10225cf7ad3c6; login_sid_t=c405bc41e6a0e367fbc86f1170fb1327; cross_origin_proto=SSL; Ugrow-G0=5b31332af1361e117ff29bb32e4d8439; wb_view_log=1366*7681; _s_tentry=-; Apache=7137970351255.176.1552525775123; ULV=1552525775142:4:1:1:7137970351255.176.1552525775123:1534248239399; TC-Page-G0=1e758cd0025b6b0d876f76c087f85f2c; UOR=,,www.baidu.com; WBtopGlobal_register_version=ae9a9ec008078a68; SCF=AnGup5xUDU7wSC7Iw0RtYBnDIt0k6AVCUs_E_BG_2ehOuz-RmGl2Vkqku-rYC-j3cXoUrWAzh8w6Gs6ekkPBCPE.; SUB=_2A25xjcvODeRhGeNN7FER-SnJyzmIHXVS-roGrDV8PUNbmtAKLWrkkW9NSY28D25qV3Zw7AreRFtT9b0EkigK4K5z; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WF1.qFZbdJaTu5Vd3xATuc45JpX5K2hUgL.Fo-0S0e71KMfeh-2dJLoIpjLxK-LBo5L12qLxK.LB.-L1K.LxKqLBo2L1-zt; SUHB=0u5hKDktLY2ca5; ALF=1553135135; SSOLoginState=1552530334; un=18981651893"}
    user_id = (int)(input("输入你要爬取的微博id:"))
    page_num = (int)(input("输入你要爬取的页数:"))
    for page in range(1,page_num+1):
        url = "https://weibo.cn/%d?page=%d"%(user_id,page)
        count = 1
        print(url)
        try:
            lxml = get_html(url,cookies)
            get_comment(lxml,count)
            get_img(lxml)
        except:
            traceback.format_exc
            continue
main()

输入你要爬取的微博id:1223178222
输入你要爬取的页数:3
https://weibo.cn/1223178222?page=1
开始爬取文字微博
文字微博爬取完成
开始爬取图片微博
https://weibo.cn/1223178222?page=2
开始爬取文字微博
文字微博爬取完成
开始爬取图片微博
https://weibo.cn/1223178222?page=3
开始爬取文字微博
文字微博爬取完成
开始爬取图片微博


In [ ]:
# -*- coding: utf-8 -*-

import urllib.request
import json
import string

#定义要爬取的微博大V的微博ID
id='1223178222'

#设置代理IP
proxy_addr="122.241.72.191:808"

#定义页面打开函数
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data = use_proxy(url, proxy_addr)
    content = json.loads(data).get('data')
    # for key, values in content.get('tabsInfo').get('tabs').items():          只是取了键，没有取到值
    if content != None and content.get('tabsInfo') != None:
        for info in content.get('tabsInfo').get('tabs'):
            if info.get('tab_type') == 'weibo':
                containerid = info.get('containerid')
                return containerid

#获取微博大V账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_userInfo(id):
    url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    data = use_proxy(url, proxy_addr)
    content = json.loads(data).get('data')
    profile_image_url = content.get('userInfo').get('profile_image_url')
    description = content.get('userInfo').get('description')
    profile_url = content.get('userInfo').get('profile_url')
    verified = content.get('userInfo').get('verified')
    guanzhu = content.get('userInfo').get('follow_count')
    name = content.get('userInfo').get('screen_name')
    fensi = content.get('userInfo').get('followers_count')
    gender = content.get('userInfo').get('gender')
    urank = content.get('userInfo').get('urank')
    print("微博昵称："+name+"\n"+"微博主页地址："+profile_url+"\n"+"微博头像地址："+profile_image_url+"\n"+"是否认证："+str(verified)+"\n"+"微博说明："+description+"\n"+"关注人数："+str(guanzhu)+"\n"+"粉丝数："+str(fensi)+"\n"+"性别："+gender+"\n"+"微博等级："+str(urank)+"\n")

#获取微博内容信息,并保存到文本中，内容包括：每条微博的内容、微博详情页面地址、点赞数、评论数、转发数等
def get_weibo(id,file):
    i=1
    while True:
        url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
        weibo_url = 'https://m.weibo.cn/api/container/getIndex?type=uid&value='+id+'&containerid='+get_containerid(url)+'&page='+str(i)
        try:
            data = use_proxy(weibo_url, proxy_addr)
            content = json.loads(data).get('data')
            cards = content.get('cards')
            if len(cards) > 0:
                for j in range(len(cards)):
                    print("-----正在爬取第"+str(i)+"页,第"+str(j)+"条微博------")
                    card_type = cards[j].get('card_type')
                    if card_type == 9:
                        mblog = cards[j].get('mblog')
                        attitudes_count = mblog.get('attitudes_count')
                        comments_count = mblog.get('comments_count')
                        created_at = mblog.get('created_at')
                        reposts_count = mblog.get('reposts_count')
                        scheme = cards[j].get('scheme')
                        text = mblog.get('text')
                        with open(file, 'a', encoding='utf-8') as fh:
                            fh.write("----第"+str(i)+"页，第"+str(j)+"条微博----"+"\n")
                            fh.write("微博地址："+str(scheme)+"\n"+"发布时间："+str(created_at)+"\n"+"微博内容："+text+"\n"+"点赞数："+str(attitudes_count)+"\n"+"评论数："+str(comments_count)+"\n"+"转发数："+str(reposts_count)+"\n")
                i += 1
            else:
                break
        except Exception as e:
            print(e)
            pass

if __name__=="__main__":
    file = id+".txt"
    get_userInfo(id)
    get_weibo(id, file)

微博昵称：胡歌
微博主页地址：https://m.weibo.cn/u/1223178222?uid=1223178222&luicode=10000011&lfid=1005051223178222
微博头像地址：https://tvax3.sinaimg.cn/crop.0.0.1125.1125.180/48e837eely8fukyqrf4ijj20v90v9mzy.jpg
是否认证：True
微博说明：🎈                                                                                                                                🏃🏻
关注人数：675
粉丝数：66374964
性别：m
微博等级：44

-----正在爬取第1页,第0条微博------
-----正在爬取第1页,第1条微博------
-----正在爬取第1页,第2条微博------
-----正在爬取第1页,第3条微博------
-----正在爬取第1页,第4条微博------
-----正在爬取第1页,第5条微博------
-----正在爬取第1页,第6条微博------
-----正在爬取第1页,第7条微博------
-----正在爬取第1页,第8条微博------
-----正在爬取第1页,第9条微博------
-----正在爬取第1页,第10条微博------
-----正在爬取第2页,第0条微博------
-----正在爬取第2页,第1条微博------
-----正在爬取第2页,第2条微博------
-----正在爬取第2页,第3条微博------
-----正在爬取第2页,第4条微博------
-----正在爬取第2页,第5条微博------
-----正在爬取第2页,第6条微博------
-----正在爬取第2页,第7条微博------
-----正在爬取第2页,第8条微博------
-----正在爬取第3页,第0条微博------
-----正在爬取第3页,第1条微博------
-----正在爬取第3页,第2条微博------
-----正在爬取第3页,第3条微博------
-----正在爬取第3页,第4条微博-----

-----正在爬取第34页,第0条微博------
-----正在爬取第34页,第1条微博------
-----正在爬取第34页,第2条微博------
-----正在爬取第34页,第3条微博------
-----正在爬取第34页,第4条微博------
-----正在爬取第34页,第5条微博------
-----正在爬取第34页,第6条微博------
-----正在爬取第34页,第7条微博------
-----正在爬取第34页,第8条微博------
-----正在爬取第35页,第0条微博------
-----正在爬取第35页,第1条微博------
-----正在爬取第35页,第2条微博------
-----正在爬取第35页,第3条微博------
-----正在爬取第35页,第4条微博------
-----正在爬取第35页,第5条微博------
-----正在爬取第35页,第6条微博------
-----正在爬取第35页,第7条微博------
-----正在爬取第35页,第8条微博------
-----正在爬取第35页,第9条微博------
-----正在爬取第36页,第0条微博------
-----正在爬取第36页,第1条微博------
-----正在爬取第36页,第2条微博------
-----正在爬取第36页,第3条微博------
-----正在爬取第36页,第4条微博------
-----正在爬取第36页,第5条微博------
-----正在爬取第36页,第6条微博------
-----正在爬取第36页,第7条微博------
-----正在爬取第36页,第8条微博------
-----正在爬取第36页,第9条微博------
